ClearML Template

In [2]:
!pip install clearml

from clearml import Task, Dataset

import pandas as pd
import os
import time


os.environ["CLEARML_API_ACCESS_KEY"] = "BF3OQG84RHSYHO86OIYII6Y7VVVMDV"
os.environ["CLEARML_API_SECRET_KEY"] = "7qQDrf96hXOsbmEio_kUbkEjUqqKaMyq4HB4K1fzOWHjKPelw2HEJdB3RrUvh4FZlA8"
os.environ["CLEARML_API_HOST"] = "https://api.clear.ml"
os.environ["CLEARML_WEB_HOST"] = "https://app.clear.ml"
os.environ["CLEARML_FILES_HOST"] = "https://files.clear.ml"

In [3]:
testDataPath = Dataset.get(dataset_id="a5958fdaf8cd449bbadc2bf57e5e797b").get_local_copy()
print(testDataPath)

/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b


In [4]:
test_val_train_folder=os.listdir(testDataPath)

print(testDataPath+"/"+test_val_train_folder[0])
print(testDataPath+"/"+test_val_train_folder[1])
print(testDataPath+"/"+test_val_train_folder[2])

/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/train_filtered.tsv
/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/valid_filtered.tsv
/root/.clearml/cache/storage_manager/datasets/ds_a5958fdaf8cd449bbadc2bf57e5e797b/test_filtered.tsv


In [5]:
# wczytujemy do DataFrame
columns = [
    "id", "label", "statement", "subjects", "speaker", "speaker_job", "state_info",
    "party_affiliation", "barely_true", "false", "half_true",
    "mostly_true", "pants_on_fire", "context"
]

testDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[0], sep='\t', names=columns, header=None)
valDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[1], sep='\t', names=columns, header=None)
trainDF = pd.read_csv(testDataPath+"/"+test_val_train_folder[2], sep='\t',names=columns, header=None )

df_train = trainDF
df_valid = valDF
df_test = testDF

In [6]:
testDF.head(1)

,id,label,statement,subjects,speaker,speaker_job,state_info,party_affiliation,barely_true,false,half_true,mostly_true,pants_on_fire,context
0,2635.json,False,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer


# Pre processing


### Importy

In [7]:
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import spacy

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Stop words

In [8]:
stop_words = set(stopwords.words('english'))

In [9]:
#2 wersja z użyciem spacy

# nlp = spacy.load('en_core_web_sm')

#można tez połączyć te dwie metody robiąc listę każdej i potem .concat()

### Czyszczenie tekstu

In [10]:
def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove all digits from text
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'<[^>]+>', '', text)  # remove html tags
    text = re.sub(r'<[^>]*>', '', text)  # additional html tag removal
    emojis = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)  # find emojis
    text = re.sub(r'[\W]+', ' ', text) + ' '.join(emojis).replace('-', '')
    text = text.strip() # remove
    return text

### Lematyzacja

In [11]:
lemmatizer = WordNetLemmatizer()

### Stemming

In [12]:
porter = PorterStemmer()

### Tokenizacja + poprzednie funkcje

In [13]:
def custom_tokenizer(text, cfg):
    text = clean_text(text)
    tokens = word_tokenize(text)

    if cfg["lemmatization"]:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    if cfg["stemming"]:
        tokens = [porter.stem(token) for token in tokens]
    if cfg["remove_stopwords"]:
        tokens = [t for t in tokens if t not in stop_words]
    tokens = [t for t in tokens if len(t) >= cfg["min_token_length"]]

    if cfg["concat_to_sentence"]:
        return " ".join(tokens)
    return tokens


# Metoda Embedding

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

def bowSklearn(train, val, test):
  vectorizer = CountVectorizer(binary=True)
  X = vectorizer.fit_transform(train) # macierz z występowaniami
  z = vectorizer.transform(val)
  y = vectorizer.transform(test)
  return X.toarray(), z.toarray(), y.toarray()

# Metoda Klasyfikacji

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

def getModel(shape):
  model = Sequential([
    Input(shape),
    Dense(8, activation='relu'),
    Dropout(0.1),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
  ])

  model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
  )

  return model

# Wywołanie Taska

In [31]:
task_name = "Testowanko BoW + Model"

# Tworzenie taska
task = Task.init(project_name="FakeNewsDetection", task_name=task_name)

# config do preprocesingu
cfg_preprocessing ={
    "tokenizer": True,
    "lemmatization": True,
    "stemming": False,
    "remove_stopwords": True,
    "min_token_length": 3, # wywala slowa któtsze niz 3
    "concat_to_sentence": True # czy łaczymy w zdania (True) czy zostawimy jako wyrazy (False)
}

# preprocessing
df_train["tokens"] = df_train["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_valid["tokens"] = df_valid["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))
df_test["tokens"] = df_test["statement"].apply(lambda x: custom_tokenizer(x, cfg_preprocessing))


# wyswietl przyklad preprocesingu
print(df_train[["statement", "tokens"]].head(3))


# metoda
start_time = time.time()

# Tutaj kod ..... = history = model.fit(...)

X_train, X_val, X_test = bowSklearn(df_train["tokens"], df_valid["tokens"], df_test["tokens"])

y_train = df_train["label"]
y_val = df_valid["label"]
y_test = df_test["label"]

model = getModel(X_train[1].shape)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_data=(X_val, y_val)
)

test_loss, test_accuracy = model.evaluate(X_test, y_test)

end_time = time.time()
training_duration = end_time - start_time

# Tutaj kod ..... = test_loss, test_accuracy = model.evaluate(X_test, y_test)

logger = task.get_logger()
for epoch in range(len(history.history['loss'])):
    logger.report_scalar("Loss", "train", value=history.history['loss'][epoch], iteration=epoch)
    logger.report_scalar("Loss", "val", value=history.history['val_loss'][epoch], iteration=epoch)
    logger.report_scalar("Accuracy", "train", value=history.history['accuracy'][epoch], iteration=epoch)
    logger.report_scalar("Accuracy", "val", value=history.history['val_accuracy'][epoch], iteration=epoch)

logger.report_scalar("Evaluation", "Test Loss", value=test_loss, iteration=0)
logger.report_scalar("Evaluation", "Test Accuracy", value=test_accuracy, iteration=0)

logger.report_scalar("Training", "Duration (s)", value=training_duration, iteration=0)

# Zamykanie taska
task.close()

ClearML Task: created new task id=f20d38c7c05b4789a9a6f951ae3f6095
ClearML results page: https://app.clear.ml/projects/25cc2e9801f0421ba3bf3ef6bcb791c7/experiments/f20d38c7c05b4789a9a6f951ae3f6095/output/log
                                                              statement  \
0  Building a wall on the U.S.-Mexico border will take literally years.   
1       Wisconsin is on pace to double the number of layoffs this year.   
2                   Says John McCain has done nothing to help the vets.   

                                              tokens  
0  building wall usmexico border take literally year  
1           wisconsin pace double number layoff year  
2              say john mccain done nothing help vet  
Epoch 1/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - accuracy: 0.5145 - loss: 0.6940 - val_accuracy: 0.4421 - val_loss: 0.6945
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5443 - loss: 0.6926 - val_accuracy: 0.4583 - val_loss: 0.6943
Epoch 3/50
15/15 